In [1]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import scipy.stats as stats
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#configuraciones generales
pd.set_option('display.max_columns', None)

randon_generator = np.random.default_rng(45)

merge_df = True

In [4]:
# Importamos el dataset de train
df_train = pd.read_csv('data/train.csv')
print(df_train.shape)
#Importamos el dataset de test
df_test = pd.read_csv('data/test.csv')
print(df_test.shape)

(103904, 25)
(25976, 25)


In [ ]:

# Unimos ambos datasets para separarlos por nuestro lado
if merge_df:
    df_air = pd.concat([df_train, df_test]).sample(frac=1, random_state=randon_generator)
else:
    df_air = df_train

# O podemos utilz
# Borramos la columna Unnamed
df_total.drop(columns='Unnamed: 0', inplace=True)
